# Noise mitigation via zero-noise extrapolation

Here we will use the functions developed in `./noise_mitigation_utils.py` to demonstrate how this technique can improve
 results when running on a noisy device.

In [ ]:
from noise_mitigation_utils import NoiseMitigation
from utility import *
import numpy as np
import qiskit
import cirq
import tequila as tq

Let's take an ansatz trained in S3 and calculate the expectation value on a noisy device.

In [ ]:
xyz_data = get_molecular_data('h2o', geometry=0.75, xyz_format=True)

basis = '6-31g'
active = {'B1':[0,1], 'A1':[2,3]}
h2o = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis, active_orbitals = active)
exact = h2o.compute_energy(method='fci')
hf_reference = hf_occ(2*h2o.n_orbitals, h2o.n_electrons)
ham = h2o.make_hamiltonian()
n_ents = ham.n_qubits
ranked_entangler_groupings = generate_QCC_gradient_groupings(ham.to_openfermion(),
                                                             2*h2o.n_orbitals,
                                                             hf_reference,
                                                             cutoff=1.e-6)
entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*h2o.n_orbitals)

for l in range(1):
    if l == 0:
        qansatz = construct_QMF_ansatz(ham.n_qubits, l)
    else:
        qansatz += construct_QMF_ansatz(ham.n_qubits, l)
    qansatz += construct_QCC_ansatz(entanglers)
E = tq.ExpectationValue(H=ham, U=qansatz)
init = {k : np.random.rand() for k in E.extract_variables()}
noiseless = tq.minimize(objective=E, initial_values=init, method='adam', tol=1.0e-6, print_level=1)